In [1]:
# import libraries
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from utilities import test_classification
from utilities import set_seed
from utilities import import_data
from utilities import split_data
from utilities import CreateDataset

In [2]:
def generate_two_spirals(n_points, noise=.5):
    n = np.sqrt(np.random.rand(n_points, 1)) * 780 * (2 * np.pi) / 360
    d1x = -np.cos(n) * n + np.random.rand(n_points, 1) * noise
    d1y = np.sin(n) * n + np.random.rand(n_points, 1) * noise
    return (np.vstack((np.hstack((d1x, d1y)), np.hstack((-d1x, -d1y)))),
            np.hstack((np.zeros(n_points), np.ones(n_points))))

X, labels = generate_two_spirals(1000)

data = pd.DataFrame({'0': labels, '1': X[:, 0], '2': X[:, 1]})
plt.scatter(X[:,0], X[:,1], c=labels)

In [3]:
class CasperNet(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_neurons=0):
        super(CasperNet, self).__init__()
        self.total_neurons = input_dim + output_dim
        self.output_dim = output_dim
        self.hidden_dim = input_dim # dimension of the hidden layer (dimension of the cascased input)
        self.output_layer = nn.Linear(input_dim, output_dim) # start with minimal network
        self.hidden_layers = nn.ModuleList() # maintain a list of hidden layer
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
        for i in range(num_hidden_neurons):
            self.add_neuron()

    def forward(self, x):
        
        # loop through all hidden layer, cascade to inputs
        for layer in self.hidden_layers:
            hidden_output = self.sigmoid(layer(x)) # output of a hidden unit
            x = torch.cat((x, hidden_output), dim=1)  # cascade the output to the previous inputs
        
        return self.output_layer(x)

    
    def add_neuron(self):
        new_neuron = nn.Linear(self.hidden_dim, 1)
        torch.nn.init.xavier_uniform_(new_neuron.weight)
        self.hidden_dim += 1 # update hidden layer dimension
        self.total_neurons += 1
        self.hidden_layers.append(new_neuron) # add the candidate to the hidden unit list
        
        
        # Preserve old weights and biases
        old_weights = self.output_layer.weight.data
        old_biases = self.output_layer.bias.data
    
        # Create new output layer
        self.output_layer = nn.Linear(self.hidden_dim, self.output_dim)
    
        # Assign old weights and biases back
        self.output_layer.weight.data[:, :-1] = old_weights
        self.output_layer.bias.data = old_biases
        
        # Initialize the new weights (last column) - using xavier initialization as an example
        nn.init.xavier_uniform_(self.output_layer.weight.data[:, -1].unsqueeze(1))

def train_network(model, train_data, optimiser, P, verbose=False):
    max_epoch = 5000 # just in case it never stop
    threshold = 0.01
    criterion = nn.CrossEntropyLoss()
    epoch = 1
    prev_train_loss = float('inf')
    train_loss = 0
    
    threshold_P = 0 # how many times does the model fall below the threshold
    threshold_P_max = 15 + model.total_neurons * P # stop training if threshold_P exceed this value
    
    while True:
        correct = 0 # total correct predictions      
        
        train_input = train_data[:, 1:]
        train_target = train_data[:, 0]
        inputs = torch.Tensor(train_input).float()
        labels = torch.Tensor(train_target).long()
        
        optimiser.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()
        train_loss = loss.item()
            
        # calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct += sum(predicted.data.numpy() == labels.data.numpy())
        accuracy = correct / len(train_data) * 100
        
        
        if prev_train_loss == float('inf'):
            percent = 1
        elif prev_train_loss == 0:
            break
        elif accuracy == 100:
            break
        else:
            percent = abs((prev_train_loss - train_loss) / prev_train_loss)
        
        # print loss and accuracy
        if verbose:
            if (epoch % 50) == 0 or epoch - 1 == 0:
                print(f'Epoch {epoch}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f} %')
        
        
        if percent < threshold:
            threshold_P += 1
        else:
            threshold_P = 0
        
        # prevent infinite loop
        if epoch >= max_epoch:
            break 
        
        if threshold_P >= threshold_P_max:
            break                          
        
        prev_train_loss = train_loss
        epoch += 1
    if verbose:
        print(f'Training stop at epoch {epoch} with loss = {train_loss:.4f}')
    return train_loss


def train(model, train_data, max_hidden_neurons, P, D, T, lr, verbose=False):
    
    best_model = None
    best_score = np.inf
    best_hidden_num = 0
    
    # define optimiser parameter
    def update_optimiser(model):
        optimiser_parameters = get_optimiser_parameters(model, lr[0], lr[1], lr[2])
        return CasperRprop(optimiser_parameters,D=D,T=T)
    
    hidden_neurons = 0
    
    # keep adding neuron to the network
    while(True):
        # train the network
        optimiser = update_optimiser(model)
        loss = train_network(model, train_data, optimiser, P, verbose)
        
        # save the best model
        if loss < best_score:
            best_model = model.state_dict()
            best_hidden_num = hidden_neurons
            best_score = loss
        
        
        if hidden_neurons >= max_hidden_neurons:
            break
        
        # add an neuron
        model.add_neuron()
        hidden_neurons += 1
    
    # net = CasperNet(2, 2, (len(best_model) // 2 - 1))
    # net.load_state_dict(best_model)
    
    if verbose:
        print(f"\nModel with hidden neuron = {best_hidden_num} is the best with loss = {best_score}\n")
    
    return best_model, best_score

class CasperRprop(optim.Rprop):
    def __init__(self, params, lr=1e-2, etas=(0.5, 1.2), step_sizes=(1e-6, 50), D=0.01, T=1, *args, **kwargs):
        super(CasperRprop, self).__init__(params, lr, etas, step_sizes, *args, **kwargs)
        
        # Additional parameters for weight decay
        self.D = D
        self.T = T
        self.H_epoch = 0  # Initialize epoch count
        
    def step(self, closure=None):
        # Increment epoch count
        self.H_epoch += 1
        
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                
                grad = p.grad.data
                state = self.state[p]
                
                # Your custom weight decay term
                weight_decay_term = - self.D * torch.sign(p.data) * p.data**2 * (2**(-self.T * self.H_epoch))
                
                # Apply the weight decay to the gradient
                grad.add_(weight_decay_term)
                
                # Rest of the Rprop logic remains the same...

        # Call the parent's step method to apply the modified gradient
        super(CasperRprop, self).step(closure)
        
        return loss
    
def get_optimiser_parameters(model, lr_l1, lr_l2, lr_l3):
    param_l1 = []
    param_l3 = []
    
    param_l2 = [list(model.output_layer.parameters())[0]]
    param_l3.append(list(model.output_layer.parameters())[1]) # bias of output neurons is L3
    
    if len(model.hidden_layers) != 0:
        param_l1 = [list(model.hidden_layers[-1].parameters())[0]] # weights of the new hidden neuron is L1
        param_l3.extend(list(model.hidden_layers[:-1].parameters())) # weights and bias of the other hidden neurons are L3
        param_l3.append(list(model.hidden_layers[-1].parameters())[1]) # bias of the new hidden neuron is L3
        

    
    # Create parameter groups
    parameters = [
        {"params": param_l1, "lr": lr_l1},
        {"params": param_l2, "lr": lr_l2},
        {"params": param_l3, "lr": lr_l3},
    ]
    
    return parameters

# test the model (modified from lab 2)
def test_classification(model, train_data, test_data):
    model.eval()
    
    # test on train set
    train_input = train_data[:, 1:]
    train_target = train_data[:, 0]
    inputs = torch.Tensor(train_input).float()
    targets = torch.Tensor(train_target).long()
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    print("Confusion matrix for training:")
    print(confusion_matrix(targets.data, predicted.cpu().long().data))
    
    plt.scatter(inputs[:,0], inputs[:,1], c=predicted)

    # test on test set
    test_input = test_data[:, 1:]
    test_target = test_data[:, 0]
    inputs = torch.Tensor(test_input).float()
    targets = torch.Tensor(test_target).long()
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    print("Confusion matrix for testing:")
    print(confusion_matrix(targets.data, predicted.cpu().long().data))
    
    # print test accuracy
    total = predicted.size(0)
    correct = predicted.cpu().data.numpy() == targets.data.numpy()
    print('Testing Accuracy: %.2f %%' % (100 * sum(correct)/total))
    
    plt.scatter(test_input[:,0], test_input[:,1], c=predicted)

Epoch 1, Loss: 1.9650, Accuracy: 44.8750 %
Epoch 50, Loss: 0.6679, Accuracy: 55.7500 %
Training stop at epoch 73 with loss = 0.6679
Epoch 1, Loss: 0.8741, Accuracy: 51.1250 %
Epoch 50, Loss: 0.6393, Accuracy: 59.5625 %
Training stop at epoch 71 with loss = 0.6374
Epoch 1, Loss: 0.6370, Accuracy: 56.8750 %
Epoch 50, Loss: 0.6308, Accuracy: 55.5000 %
Training stop at epoch 79 with loss = 0.6289
Epoch 1, Loss: 0.6676, Accuracy: 66.3750 %
Epoch 50, Loss: 0.6259, Accuracy: 54.3750 %
Training stop at epoch 90 with loss = 0.6215
Epoch 1, Loss: 0.6491, Accuracy: 61.8125 %
Epoch 50, Loss: 0.6012, Accuracy: 63.5000 %
Epoch 100, Loss: 0.5787, Accuracy: 72.0625 %
Training stop at epoch 105 with loss = 0.5756
Epoch 1, Loss: 0.6612, Accuracy: 68.6250 %
Epoch 50, Loss: 0.5314, Accuracy: 73.0625 %
Epoch 100, Loss: 0.4914, Accuracy: 72.5625 %
Training stop at epoch 120 with loss = 0.4717
Epoch 1, Loss: 0.5021, Accuracy: 73.8750 %
Epoch 50, Loss: 0.4458, Accuracy: 75.3125 %
Epoch 100, Loss: 0.3575, Accu

In [4]:
# make results determinstic
seed = 4660
if seed != None:
    set_seed(seed)
        
# Define hyperparameter
hidden_neurons = 10
P = 10
lr = [0.001, 0.005, 0.2]
D = 0
T = 1

num_classes, input_size = 2, 2

# randomly split data into training set (80%) and testing set (20%)
train_data, test_data, _, _ = train_test_split(data, data.iloc[:,0], test_size=0.2, random_state=seed)
train_data, test_data = np.array(train_data), np.array(test_data)

# initialise network
casper_net = CasperNet(input_size, num_classes)

# train the model
train(casper_net, train_data, hidden_neurons, P, D, T, lr, verbose=True)

# test the model
test_classification(casper_net, train_data, test_data)

In [5]:
# validate a specific hyperparameter
def validator(data, hidden_neuron=10, P = 10, lr = [0.001, 0.005, 0.2], D = 0.01, T = 1, average_scores=True):
    data = np.array(data)
    input_dim = data.shape[1] - 1
    overall_score = 0 # average score across all seeds
    all_scores = []
    
    for seed in seeds:
        score = 0 # 10-fold validation score when using this seed
        set_seed(seed) # set a seed
        kf = KFold(n_splits=10, shuffle=True, random_state=seed) # split data
        fold_count = 1
        
        for train_index, val_index in kf.split(data):
            # obtain training and validation set
            data_train, data_val = data[train_index], data[val_index]
            
            # initialise network
            casper_net = CasperNet(input_dim, num_classes)
            
            # train the model
            train(casper_net, data_train, hidden_neuron, P, D, T, lr)
            
            # set to evaluation mode
            casper_net.eval()
            
            # compute validation accuracy
            inputs = torch.Tensor(data_val[:, 1:]).float()
            targets = torch.Tensor(data_val[:, 0]).long()
            outputs = casper_net(inputs)
            _, predicted = torch.max(outputs, 1)
            total = predicted.size(0)
            correct = predicted.cpu().data.numpy() == targets.data.numpy()
            accuracy = sum(correct) / total * 100
            print(f'Seed: {seed}   Fold: {fold_count}/10   Accuracy: {accuracy:.4f} %', end = '\r')
            score += accuracy
            fold_count += 1
            all_scores.append(accuracy)
        
        score /= 10
        overall_score += score
    
    overall_score /= len(seeds)
    
    if average_scores:
        return overall_score
    else:
        return all_scores

97.74375 3.2634277895335755Accuracy: 100.0000 %


In [6]:
seeds = [1130,1013,1005,1101,1140,1218,1600,1014]
scores_spiral = validator(data, hidden_neuron=10, P = 10, lr = [0.001, 0.005, 0.2], D = 0, T = 1, average_scores=False)
print(np.mean(scores_spiral),np.std(scores_spiral))

# print the distribution
plt.hist(scores_spiral, color='lightgreen', ec='black', bins=20)
plt.show()